In [2]:
# from flask_other.app import app
from flask_socketio import disconnect as dc
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from flask_socketio import disconnect
from threading import Lock
import os,sys

app=Flask("demo")
from flask import  render_template,request
import time
import re

import pandas as pd
import numpy as np
import time
import re
import requests
import urllib

import atexit

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger

logger = Logger('Websocket-Flask',level=ENV.NODE_LOG_LEVEL.value).logger



##################################
async_mode = None
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)

thread = None
thread_lock = Lock()
clients = []
sentences = []

name_space = '/chat'

@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace=name_space)
def client_msg(msg):  
    uid = request.sid
    sentence = msg.get('data')
    sentence = decode(sentence)
    sentences.append(sentence)
    response = cache.chat(uid, sentence)
    if response == 'end':
        socketio.emit('my_response',{'data':'感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    elif response is None:
        socketio.emit('my_response',{'data':'当前会话已经过期，感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    else:
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)

def decode(msg):
    print('before decode: {}'.format(msg))
    msg = re.sub(r'%u', r'\u', msg)
    msg = urllib.parse.unquote(msg)
    msg = msg.encode('latin-1').decode('unicode_escape')
    
    print('after decode: {}'.format(msg))
    return msg

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace=name_space)
    
def disconnect_frontend(uid):
    socketio.emit('my_response',{'status':'disconnected'},room = uid, namespace=name_space)


@socketio.on('disconnect',namespace=name_space)
def disconnect():
    uid = request.sid
    cache.remove_session(uid)
    leave_room(uid)
    dc()
    logger.info('{} has been disconnected from API'.format(uid))
    
    
@socketio.on('connect',namespace=name_space)
def connect():
    
    uid = request.sid
    clients.append(uid)
    logger.info('{} is trying to connect!'.format(uid))
    
    join_room(uid)
    if cache.create_session(uid):
        logger.info('{} join connection successfully'.format(uid))
        response = cache.chat(uid, '')
        if response is not None:
            socketio.emit('my_response',{'data':response},room = uid,namespace=name_space) #the first sentence
            return None
        
    else:
        logger.info('{} cannot join connection'.format(uid))
        socketio.emit('my_response',{'data':'server busy. please click new conv'},room = uid,namespace=name_space) 
        disconnect_frontend(uid)
    



In [ ]:
print('http://10.0.24.31:8899/')
print('http://0.0.0.0:6006/')
scheduler = BackgroundScheduler()
scheduler.start()
scheduler.add_job(
    func=cache.purge_inactive,
    trigger=IntervalTrigger(seconds=3),
    id='purge_cache',
    name='purge_inactive',
    replace_existing=True)
# Shut down the scheduler when exiting the app
atexit.register(lambda: scheduler.shutdown())

socketio.run(app,'0.0.0.0',6006)

http://10.0.24.31:8899/
http://0.0.0.0:6006/


2018-07-30 20:05:46,501 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:95 - MSG:35e254a4683d4e8698ba676cbdd8be9c is trying to connect!
2018-07-30 20:05:46,512 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:206 - MSG:profile is None. The default demo profile will be loaded!
2018-07-30 20:05:46,514 - INFO - CLASS:PF- METHOD:_load_default -LINE:227 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-30 20:05:46,552 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:261 - MSG:Load profile Upper bound successfully!
2018-07-30 20:05:46,556 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:268 - MSG:Load profile Lower bound successfully!


Time Zone is set from ENV: Asia/Shanghai


2018-07-30 20:05:47,123 - INFO - CLASS:Cache- METHOD:create_session -LINE:53 - MSG:New session was created: 35e254a4683d4e8698ba676cbdd8be9c
2018-07-30 20:05:47,124 - INFO - CLASS:Cache- METHOD:create_session -LINE:54 - MSG:Remaining session number is: 999
2018-07-30 20:05:47,125 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:99 - MSG:35e254a4683d4e8698ba676cbdd8be9c join connection successfully
2018-07-30 20:05:47,128 - INFO - CLASS:Cache- METHOD:chat -LINE:86 - MSG:receive message from user: 35e254a4683d4e8698ba676cbdd8be9c ====================
2018-07-30 20:05:47,129 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:460 - MSG:Current node name is s0
2018-07-30 20:05:47,130 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:465 - MSG:Output label is 0
2018-07-30 20:05:47,138 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:480 - MSG:Next node name is cf_s1_n1_identity_q.
2018-07-30 20:05:47,140 - INFO - CLASS:Cache- METHOD:chat -LINE:90 - MSG:processing messages for user 35e254a46

before decode: %u662F%u7684
after decode: 是的


2018-07-30 20:06:07,666 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:95 - MSG:81e3e577244d4fe583069d1925e9840e is trying to connect!
2018-07-30 20:06:07,669 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:206 - MSG:profile is None. The default demo profile will be loaded!
2018-07-30 20:06:07,669 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:206 - MSG:profile is None. The default demo profile will be loaded!
2018-07-30 20:06:07,671 - INFO - CLASS:PF- METHOD:_load_default -LINE:227 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-30 20:06:07,671 - INFO - CLASS:PF- METHOD:_load_default -LINE:227 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-30 20:06:07,687 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:261 - MSG:Load profile Upper bound successfully!
2018-07-30 20:06:07,687 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:261 - MSG:Load profile Upper bound successfully!
2018-07-30 20:06:07,689 - INFO - CLASS:PF- METHOD:_loadUpLowBou

Time Zone is set from ENV: Asia/Shanghai


2018-07-30 20:06:08,271 - INFO - CLASS:Cache- METHOD:create_session -LINE:53 - MSG:New session was created: 81e3e577244d4fe583069d1925e9840e
2018-07-30 20:06:08,278 - INFO - CLASS:Cache- METHOD:create_session -LINE:54 - MSG:Remaining session number is: 998
2018-07-30 20:06:08,279 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:99 - MSG:81e3e577244d4fe583069d1925e9840e join connection successfully
2018-07-30 20:06:08,282 - INFO - CLASS:Cache- METHOD:chat -LINE:86 - MSG:receive message from user: 81e3e577244d4fe583069d1925e9840e ====================
2018-07-30 20:06:08,286 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:460 - MSG:Current node name is s0
2018-07-30 20:06:08,286 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:460 - MSG:Current node name is s0
2018-07-30 20:06:08,294 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:465 - MSG:Output label is 0
2018-07-30 20:06:08,294 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:465 - MSG:Output label is 0
2018-07-30 20:06:08,302 -

before decode: %u662F%20
after decode: 是 


2018-07-30 20:06:50,865 - INFO - CLASS:DB- METHOD:insert -LINE:73 - MSG:6 was inserted into collection: chat_debug
2018-07-30 20:06:50,869 - INFO - CLASS:Cache- METHOD:remove_session -LINE:79 - MSG:35e254a4683d4e8698ba676cbdd8be9c session is inactive, it has been removed!
2018-07-30 20:06:50,927 - ERROR - CLASS:Cache- METHOD:remove_session -LINE:70 - MSG:Fail to save cache for uid: 
2018-07-30 20:06:50,991 - INFO - CLASS:Websocket-Flask- METHOD:disconnect -LINE:87 - MSG:35e254a4683d4e8698ba676cbdd8be9c has been disconnected from API
2018-07-30 20:06:50,993 - INFO - CLASS:Websocket-Flask- METHOD:disconnect -LINE:87 - MSG:35e254a4683d4e8698ba676cbdd8be9c has been disconnected from API
2018-07-30 20:07:12,278 - INFO - CLASS:Cache- METHOD:inform_inactive -LINE:155 - MSG:81e3e577244d4fe583069d1925e9840e is inactive. Just inform that user
2018-07-30 20:08:15,278 - INFO - CLASS:Cache- METHOD:inform_inactive -LINE:155 - MSG:81e3e577244d4fe583069d1925e9840e is inactive. Just inform that user
20

In [1]:
import time
import gc
import sys, os
import datetime as dt

ENV_PATH = '../../../ENV/'
LOG_PATH = '../../../Lib/'
sys.path.append(ENV_PATH)
sys.path.append(LOG_PATH)
from env import ENV
from LOG import Logger
from MGODB import DB



class Cache:
    def __init__(self, model_dict,max_session=1000,debug=False,host=None,port=None,enableDB=False):
        self.max_session = 1000
        self.inform_interval = 60
        self.inactive_maxlength = 150
        #{'uid': {'strategy': Tree(), 'time_response': <time>, 'time_inform': <>}
        self.active_session = {}
        self.model_dict = model_dict
        self.debug=debug
        self.enableDB = enableDB
        self.log = Logger(self.__class__.__name__,level=ENV.NODE_LOG_LEVEL.value).logger
        self.db=DB(host,port,debug,True,enable=self.enableDB)
        self._print()
        
        
    def _print(self):
        self.log.info('Max num of session is: {}'.format(self.max_session))
        self.log.info('inform inacitve interval is {} seconds'.format(self.inform_interval))
        self.log.info('inactive max length is {} seconds'.format(self.inactive_maxlength))
        if self.debug:
            self.log.info('DEBUG is enabled')
        
        
    def create_session(self, uid, profile=None):
        if len(self.active_session) < self.max_session:
            self.active_session[uid] = {}
            try:
                self.active_session[uid].update({'strategy':TreeStage1(
                                                                       debug=self.debug,
                                                                       profile=profile)})
            except KeyError as e:
                self.log.error('Key {} does not exist in profile'.format(e))
                self.log.error('create session for user {} failed'.format(uid))
                return False
                
            self.active_session[uid].update({'time_response':time.time()})
            self.active_session[uid].update({'time_inform':time.time()})
            self.log.info('New session was created: {}'.format(uid))
            self.log.info('Remaining session number is: {}'.format(self.max_session-len(self.active_session)))
            return True
        else:
            return False
        
    def remove_session(self,uid):
        response = '您当前的会话超过 {} 秒没有响应，系统将关闭当前会话！如有需求，请开始新的对话！'.format(self.inactive_maxlength)
        try:
            socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        except:
            pass
        
        try:
            history = self.active_session[uid]['strategy'].cache.copy()
            self.db.insert(history)
        except:
            self.log.error('Fail to save cache for uid: ')
        try:
            disconnect_frontend(uid)
        except:
            pass
       
        
        try:
            del self.active_session[uid]
            self.log.info('{} session is inactive, it has been removed!'.format(uid))
        except KeyError:
                pass
        gc.collect()
            
    def chat(self,uid,sentence):
        if self.active_session.get(uid) is not None:
            self.log.info('receive message from user: {} ===================='.format(uid))
            response = self.active_session[uid]['strategy'].process(sentence, self.model_dict)
            self.active_session[uid]['time_response'] = time.time()
            self.active_session[uid]['time_inform'] = time.time()
            self.log.info('processing messages for user {} has been done!----------------'.format(uid))
        else:
            response = None
        
        return response
        
    
    
    def _bulk_deletes(self):
        current = time.time()
        remove_list = []
        try:
            for uid in self.active_session:
                try:
                    if current - self.active_session[uid]['time_response'] > self.inactive_maxlength:
                        remove_list.append(uid)
                except KeyError:
                    pass
        except RuntimeError as e:
            self.log.error(e)
            return False
        finally:
            # delete
            for uid in remove_list:              
                self.remove_session(uid)
        return True
    
    def _bulk_inform(self):
        current = time.time()
        inform_list = []
        try:
            for uid in self.active_session:
                try:
                    if current - self.active_session[uid]['time_inform'] > self.inform_interval:
                        inform_list.append(uid)
                     
                except KeyError as e:
                        self.log.error(e)
                        pass
            for each in inform_list:
                self.inform_inactive(each)
            
        except RuntimeError:
            
            return False
        
        return True
    
    
    def purge_inactive(self):
        current = time.time()
        while True:
            if self._bulk_deletes():
                break

        while True:
            if self._bulk_inform():
                break
        
                
        
    def inform_inactive(self, uid):
        self.active_session[uid]['time_inform'] = time.time()
        response = '您有在听我说吗?请回答我刚才的问题！'
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        self.log.info('{} is inactive. Just inform that user'.format(uid))
          
################################################
import sys,os
sys.path.append('../../../MLModel/code/OneClickTraining/')
sys.path.append('../../../MLModel/code/TreeModelV2/')
from all_model_py import *
import pickle
from chatbotv1 import *



models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../../MLModel/savedModel/{}/{}.pickle'
need_set_TIMEZONE = ['CutDebt','WillingToPay','Installment','ConfirmLoan']

model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    model_dict[each_model].classify('再说一次')
    if each_model in need_set_TIMEZONE:
        model_dict[each_model].re_time._set_timeZone()
    
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()    

#################################################################
cache = Cache(
              model_dict=model_dict,debug=True,enableDB=True)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.724 seconds.
Prefix dict has been built succesfully.


Time Zone is set from ENV: Asia/Shanghai
Time Zone is set from ENV: Asia/Shanghai


2018-07-30 20:05:34,257 - INFO - CLASS:DB- METHOD:_load_client -LINE:35 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-30 20:05:34,262 - INFO - CLASS:DB- METHOD:_get_db -LINE:43 - MSG:mongodb database is: chatbotdb_debug
2018-07-30 20:05:34,265 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51 - MSG:mongodb collection is: chat_debug
2018-07-30 20:05:34,269 - INFO - CLASS:Cache- METHOD:_print -LINE:32 - MSG:Max num of session is: 1000
2018-07-30 20:05:34,273 - INFO - CLASS:Cache- METHOD:_print -LINE:33 - MSG:inform inacitve interval is 60 seconds
2018-07-30 20:05:34,276 - INFO - CLASS:Cache- METHOD:_print -LINE:34 - MSG:inactive max length is 150 seconds
2018-07-30 20:05:34,277 - INFO - CLASS:Cache- METHOD:_print -LINE:36 - MSG:DEBUG is enabled


Time Zone is set from ENV: Asia/Shanghai
Time Zone is set from ENV: Asia/Shanghai


In [2]:
t = TreeStage1(debug=False)

2018-07-24 18:19:29,223 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:206 - MSG:profile is None. The default demo profile will be loaded!
2018-07-24 18:19:29,227 - INFO - CLASS:PF- METHOD:_load_default -LINE:227 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-24 18:19:29,259 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:261 - MSG:Load profile Upper bound successfully!
2018-07-24 18:19:29,260 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:268 - MSG:Load profile Lower bound successfully!


Time Zone is set from ENV: Asia/Shanghai


In [4]:
cache.active_session

{'c822328a576e4712928e743b7a9932bd': {'strategy': <chatbotv1.TreeStage1 at 0x7f3fc7e19c18>,
  'time_response': 1532977051.660843,
  'time_inform': 1532977051.6608553}}

In [5]:
tt = cache.active_session['c822328a576e4712928e743b7a9932bd']['strategy']

In [6]:
cache.db.insert(tt.cache)

2018-07-30 18:57:45,717 - INFO - CLASS:DB- METHOD:insert -LINE:73 - MSG:2 was inserted into collection: chat_debug


In [7]:
cache.db.collection.count_documents(filter={})

2

In [12]:
cache.db.collection.count()

1

In [7]:
model_dict['IDClassifier'].classify('是 ')

{'label': 0,
 'pred_prob': array([[9.80271550e-01, 1.91362018e-02, 5.92247856e-04],
        [9.16616471e-01, 7.01026870e-02, 1.32808422e-02],
        [4.95791100e-01, 1.39347057e-01, 3.64861843e-01]]),
 'av_pred': array([0.79755971, 0.07619532, 0.12624498]),
 'other_response': None}

In [5]:
import urllib  